In [ ]:
from ipywidgets import interact, FloatSlider, widgets, fixed
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from tqdm import tqdm
import math
from scipy.optimize import curve_fit
from bleacher import simulation

## Determination of the LED pattern

In [ ]:
# Define sliders for interactive controls using ipywidgets
width_slider = FloatSlider(min=10, max=500, step=5, value=300, description='Width')
height_slider = FloatSlider(min=10, max=500, step=5, value=200, description='Height')
dx_slider = FloatSlider(min=1, max=50, step=0.5, value=25, description='dx')
dy_slider = FloatSlider(min=1, max=50, step=0.5, value=25, description='dy')

# Create an interactive plot using interact
interact(simulation.plot_grid, width=width_slider, height=height_slider, dx=dx_slider, dy=dy_slider);

## Interactive heatmap analysis

In [ ]:
### plot for single illumination
# set parameters
width = 160 # prototype: 160, v2: 300, Tsuneoka: 260,
height = 260 # prototype: 260, v2: 200, Tsuneoka: 100, 
dx = 15.5 # prototype: 15.5, v2: 12.5, Tsuneoka: 40, super: 14
dy = 17.0 # prototype: 17.0, v2: 12.5, Tsuneoka: 40, super: 14
depth = 45  # prototype: 45, xx, v2: 32, Tsuneoka: 5
phi = 20 #  prototype: 20 lm, v2: 223 lm, Tsuneoka: 4131 lm, Super: 1000?
unit = 0.001 # 0.001 for mm, 0.01 for cm, 1.0 for m

board_position = 0.0 # position of the light source in depth direction
illumination = 'lambertian' # either lambertian or point
grid_constant = 1.0 # sampling frequency in xy

In [ ]:
# Define sliders for interactive controls using ipywidgets
width_slider = FloatSlider(min=10, max=400, step=5, value=width, description='Width') # v2: 300, Tsuneoka: 260, 
height_slider = FloatSlider(min=10, max=400, step=5, value=height, description='Height') # v2: 200, Tsuneoka: 100, 
dx_slider = FloatSlider(min=1, max=50, step=0.5, value=dx, description='dx') # v2: 12.5, Tsuneoka: 40, super: 14
dy_slider = FloatSlider(min=1, max=50, step=0.5, value=dy, description='dy') # v2: 12.5, Tsuneoka: 40, super: 14
phi_slider = FloatSlider(min=1, max=5000, step=1, value=phi, description='phi') # v2: 177458, Tsuneoka: 3287345, super
depth_slider = FloatSlider(min=1, max=100, step=1, value=depth, description='Depth') # v2: 32, Tsuneoka: 5
vmax_slider = FloatSlider(min=1, max=1000000, step=10, value=300000, description='vmax')

# Create an interactive plot using interact
interact(simulation.plot_heatmap, width=width_slider, height=height_slider, dx=dx_slider, dy=dy_slider, 
         phi=phi_slider, depth=depth_slider, vmax=vmax_slider, 
         unit=fixed(unit), board_position=fixed(board_position), 
         grid_constant=fixed(grid_constant), illumination=fixed(illumination));

In [ ]:
### plot for dual illumination
# set parameters
board_position1 = 0.0
board_position2 = 100.0

# Define sliders for interactive controls using ipywidgets
width_slider = FloatSlider(min=10, max=400, step=5, value=width, description='Width') # v2: 300, Tsuneoka: 260, 
height_slider = FloatSlider(min=10, max=400, step=5, value=height, description='Height') # v2: 200, Tsuneoka: 100, 
dx_slider = FloatSlider(min=1, max=50, step=0.5, value=dx, description='dx') # v2: 12.5, Tsuneoka: 40, super: 14
dy_slider = FloatSlider(min=1, max=50, step=0.5, value=dy, description='dy') # v2: 12.5, Tsuneoka: 40, super: 14
phi_slider = FloatSlider(min=1, max=5000, step=1, value=phi, description='phi') # v2: 177458, Tsuneoka: 3287345, super
depth_slider = FloatSlider(min=1, max=100, step=1, value=depth, description='Depth') # v2: 32, Tsuneoka: 5
bp1_slider = FloatSlider(min=0, max=300, step=1, value=board_position1, description='BoardPos1') # default 0
bp2_slider = FloatSlider(min=0, max=300, step=1, value=board_position2, description='BoardPos2')
vmax_slider = FloatSlider(min=1, max=10000000, step=10, value=3000000, description='vmax')

# Create an interactive plot using interact
interact(simulation.plot_dual_heatmap, width=width_slider, height=height_slider, dx=dx_slider, dy=dy_slider, 
         phi=phi_slider, depth=depth_slider, vmax=vmax_slider, 
         unit=fixed(unit), board_position1=bp1_slider, board_position2=bp2_slider,
         grid_constant=fixed(grid_constant), illumination=fixed(illumination));